In [ ]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm

import swarm

In [ ]:
def f(x):
    return x[:, 0]**2 + x[:, 1]**2

constraints = np.array([[1.0, 2.5],
                        [10.0, 7.6]])



In [ ]:
sw = swarm.Swarm(f, 1, 2, constraints)
# gbest_list = []
# for i in tqdm(range(100)):
#     sw.update_swarm()
#     #print(sw.pbest_position)
#     gbest, gbest_position = sw.compute_gbest()
#     print(gbest)
#     gbest_list.append(gbest)
    
gbest, gbest_position = sw.compute_gbest()
(gbest, gbest_position)

In [ ]:
# plt.plot(gbest_list)

In [ ]:
# f(np.array([[0.26649261, 2.07550618],
#             [2.49591414, 2.5903221 ]]))

In [ ]:
# bool_decider = self.pbest >  func_eval
# self.pbest[bool_decider] = func_eval[bool_decider]

# pre = np.array(self.pbest_position, dtype=np.int32)

# self.pbest_position[np.where(self.pbest > func_eval)] = self.position[np.where(self.pbest >  func_eval)]

# past = np.array(self.pbest_position, dtype=np.int32)

# if (pre != past).any():
#     print('problem')

In [ ]:
lbest, lbest_position = sw.compute_lbest()

pbest = sw.pbest
pbest_position = sw.pbest_position

position = sw.position + sw.velocity
values = sw.func(position)

In [ ]:
position

In [ ]:
c_1, c_2 = np.ones(2)*0.5 + np.log(2)

U_1 = np.random.uniform(0, 1, (sw.n_particles, sw.dim))
U_2 = np.random.uniform(0, 1, (sw.n_particles, sw.dim))

proj_pbest = position + c_1 * U_1 * (pbest_position - position)
proj_lbest = position + c_2 * U_2 * (lbest_position - position) 

center = (position + proj_pbest + proj_lbest) / 3
r = np.linalg.norm(center - position, axis=1)

In [ ]:
u = np.random.normal(0, 1, (sw.n_particles, sw.dim+2))
u_norm = np.linalg.norm(u, axis=1)
u /= u_norm[:,None]
u *= r[:,None]

update = u[:,:sw.dim]
new_point = center + update

# Test for Hypersphere sampling in 2D

In [ ]:
random_samples=[]

for i in range(5000):
    u = np.random.normal(0, 1, (sw.n_particles, sw.dim+2))
    u_norm = np.linalg.norm(u, axis=1)
    u /= u_norm[:,None]
    u *= r[:,None]

    update = u[:,:sw.dim]
    random_samples.append(center + update)

random_samples = np.array(random_samples)

In [ ]:
random_samples.shape

In [ ]:
i=1
plt.plot(position[0, 0], position[0, 1], label='starting pos', marker='+', color='blue')
plt.plot(proj_pbest[0, 0], proj_pbest[0, 1], label='Proj_pbest', marker='+', color='red')
plt.plot(proj_lbest[0, 0], proj_lbest[0, 1], label='Proj_lbest', marker='o', color='green')
plt.plot(center[0, 0], center[0, 1], label='Center', marker='o', color='yellow')


# for idx, sample in enumerate(random_samples):

plt.plot(random_samples[:, 0, 0], random_samples[:, 0, 1], linestyle='None', marker='.', color='black')

plt.legend()


circle1 = plt.Circle(center[0, :] , r[0], color='purple')
plt.gca().add_patch(circle1)

plt.axis('equal')
plt.show()

# Test for Hypersphere sampling in 3D

In [ ]:
center=np.array([0, 0, 0])

In [ ]:
radius = 0.6
dim = 3

In [ ]:
def get_random_sample(r, dim):
    u = np.random.normal(0, 1, dim+2)
    u_norm = np.linalg.norm(u)
    u /= u_norm
    u *= r
    return u[:dim]

In [ ]:
data = [get_random_sample(radius, 3) for _ in range(100)]

In [ ]:
data = np.array(data)

In [ ]:
# fig = plt.figure()
# ax = fig.gca(projection='3d')

# for _, point in enumerate(data):
#     xs = point[0]
#     ys = point[1]
#     zs = point[2]
#     ax.scatter(xs, ys, zs)

# plt.show()

In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter_3d(x=data[:, 0], y=data[:, 1], z=data[:, 2])
fig.show()